In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn python-multipart


import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import nest_asyncio
from pyngrok import ngrok
import uvicorn

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel


from google.colab import userdata
SENDER_PASSWORD = userdata.get('EMAIL_PASSWORD')
NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN') 


ngrok.set_auth_token(NGROK_AUTHTOKEN) 

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


class ConfirmationRequest(BaseModel):
    name: str
    email: str


def send_confirmation_email(name: str, receiver_email: str):
    sender_email = "sanaaiemrchains93@gmail.com"
    sender_password = SENDER_PASSWORD

    msg = MIMEMultipart("alternative")
    msg["Subject"] = "Thank You for Your Demo Request!"
    msg["From"] = f"Sana AI Team <{sender_email}>"
    msg["To"] = receiver_email

    html_body = f"""
    <html>
      <body>
        <p>Dear {name},</p>
        <p>
          Thank you for your request to schedule a demo with Sana AI. We're excited to show you how our platform can help transform your healthcare communications.
        </p>
        <p>
          Your request has been successfully received. Our team is reviewing your details and will contact you within the next 1-2 working days to confirm a time that works for you.
        </p>
        <p>
          Many thanks and best wishes,
        </p>
        <p>
          <b>The Sana AI Team</b><br>
          <a href="https://emrchains.com/">emrchains.com</a>
        </p>
      </body>
    </html>
    """
    msg.attach(MIMEText(html_body, "html"))

    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()
        print(f"Confirmation email sent successfully to {receiver_email}")
        return True
    except Exception as e:
        print(f"Failed to send email: {e}")
        return False


@app.post("/send-confirmation")
async def send_confirmation(request: ConfirmationRequest):
    success = send_confirmation_email(request.name, request.email)
    if success:
        return {"success": True, "message": "Confirmation email sent successfully."}
    else:
        return {"success": False, "message": "Failed to send confirmation email."}


public_url = ngrok.connect(8000)
print(f"FastAPI server is running at: {public_url}")

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)